
## **ER Notation**
### **ER design elements:**
<img src="http://www.cse.unsw.edu.au/~cs3311/21T3/lectures/exercises/Pics/er-rel/er-symbols.png" width="60%">





### **Relationships:**
<img src="http://www.cse.unsw.edu.au/~cs3311/21T3/lectures/exercises/Pics/er-rel/cardinal.png" width="60%">

#### **N:M Relationships:**
<img src="http://www.cse.unsw.edu.au/~cs3311/21T3/lectures/exercises/Pics/er-rel/relNM.png" width="60%">

#### **1:N Relationships**
<img src="http://www.cse.unsw.edu.au/~cs3311/21T3/lectures/exercises/Pics/er-rel/rel1N.png" width="60%">

#### **1:1 Relationships**
<img src="http://www.cse.unsw.edu.au/~cs3311/21T3/lectures/exercises/Pics/er-rel/rel11.png" width="60%">



## **继承**
<img src="https://cgi.cse.unsw.edu.au/~cs3311/21T3/lectures/er-model/Pics/er-rel/inherit.png" width="60%" style="background-color:white;">

## **Types/Constants in SQL**

In [ ]:
create table notes (
    整数类型 integer, -- 4字节
    整数类型 REAL,    -- 4字节	可变精度，不精确, 6 位十进制数字精度
    任意精度类型 NUMERIC, -- NUMERIC(precision, scale) precision，总精度，可以存储的总位数 
                                                    -- scale，小数点后的位数
    字符类型 varchar,   -- varchar(n)， 有长度限制的字符串
    字符类型 char,      -- char(n)，定长字符串，长度不足则向后填充空白字符
    title text,        -- 不限长度
    -- Time-related types: DATE, TIME, TIMESTAMP, INTERVAL
    -- '2008-04-13'  '13:30:15'  '2004-10-19 10:23:54'
    -- 'Wed Dec 17 07:37:16 1997 PST'
    -- '10 minutes'  '5 days, 6 hours, 15 seconds'
    constraint PayOk check (salary > age*1000)

);
-- positive integers
CREATE DOMAIN PosInt AS integer CHECK (value > 0);

-- a UNSW course code
CREATE DOMAIN CourseCode AS char(8)
    CHECK (value ~ '[A-Z]{4}[0-9]{4}');

-- a UNSW student/staff ID
CREATE DOMAIN ZID AS integer
    CHECK (value betweem 1000000 and 9999999);

-- standard UNSW grades (FL,PS,CR,DN,HD)
CREATE DOMAIN Grade AS char(2)
    CHECK (value in ('FL','PS','CR','DN','HD'));
-- or
CREATE TYPE Grade AS ENUM ('FL','PS','CR','DN','HD');

select * from People where name like 'John%'; --John开头
select * from People where name like '%John'; --John结尾
select * from People where name like '_o%'; --第二个字母是o
select * from People where name like '%o%o%'; --有两个o
select * from People where name ilike 'john'; --无视大小写

-- name ~ '^Ja'		name begins with 'Ja'
-- name ~ '^.i'		name has 'i' as 2nd letter
-- name ~ '.*o.*o.*'	name contains two 'o's
-- name ~ 'ith$'		name ends with 'ith'
-- name ~ 'John'		name contains 'John'


## **SQL Expressions**

In [ ]:
‘+ - * / abs ceil floor power sqrt sin’  
count(属性)  -- attr列中 的行数  
sum(属性)  -- attr值的总和  
avg(属性)  -- attr值的平均值  
min/max(属性)  -- attr值的最小值/最大值  

coalesce(值1,值2,... val n)  
-- 返回第一个非空值val i  
-- 用于为空值提供“可显示”值  
例如 select coalesce(mark,'??') from Marks ...  

nullif(值1,值2)

-- return NULL如果val 1等于val 2
-- 可用于实现“逆” coalesce
例如 nullif(mark,'??')

-- 字符串匹配
insert into Stuff(x,y,s) values (2,4,'green');
DELETE FROM Likes WHERE drinker = 'Justin' AND beer = 'Sparkling Ale';
UPDATE Drinkers SET addr = 'Coogee', phone = '9665-4321' WHERE  name = 'John';
-- Updates all tuples
UPDATE Sells SET price = 6.00 WHERE  price > 6.00;
UPDATE Sells SET price = price * 1.10;

## **SQL Queries**

In [ ]:
-- SELECT
SELECT [ ALL | DISTINCT [ ON ( expression [, ...] ) ] ]
    [ * | expression [ [ AS ] output_name ] [, ...] ]
    [ FROM from_item [, ...] ]
    [ { JOIN | LEFT OUTER JOIN | RIGHT OUTER JOIN | FULL OUTER JOIN} join_item on condition ]
    [ WHERE condition ]
    [ GROUP BY grouping_element [, ...] ]
    [ HAVING condition ]
    [ { UNION | INTERSECT | EXCEPT } [ ALL | DISTINCT ] select ]
    [ ORDER BY expression [ ASC | DESC | USING operator ] [ NULLS { FIRST | LAST } ] [, ...] ]
    [ LIMIT { count | ALL } ]
    [ OFFSET start [ ROW | ROWS ] ]

In [ ]:
-- The IN Operator
SELECT name, brewer
FROM   Beers
WHERE  name IN
        (SELECT beer
        FROM   Likes
        WHERE  drinker = 'John');

-- The EXISTS Operator
SELECT name, brewer
FROM   Beers b1
WHERE  NOT EXISTS
        (SELECT *
        FROM   Beers b2
        WHERE  b2.brewer = b1.brewer AND b2.name <> b1.name);

-- Quantifiers ANY and ALL
SELECT beer
FROM   Sells
WHERE  price >= ALL(SELECT price FROM sells);

-- Partitions
-- Example: show each city with daily temperature and temperature range
-- Schema: Weather(city,date,temperature)
SELECT city, date, temperature
        min(temperature) OVER (PARTITION BY city) as lowest,
        max(temperature) OVER (PARTITION BY city) as highest
FROM   Weather;
-- Output: Result(city, date, temperature, lowest, highest)

In [ ]:
-- WITH- 子句抽象子查询
-- 使用定义复杂查询 WITH:
WITH CourseMarksWithAvg AS 
    (SELECT course, student, mark, 
            avg(mark) OVER (PARTITION BY course) 
    FROM Enrolments) 
SELECT course, student, mark, avg 
FROM CourseMarksWithAvg 
WHERE mark < avg;
-- 避免定义视图的需要。
WITH   Name1(a,b,c) AS (Query1),
        Name2 AS (Query2), ...
SELECT attributes
FROM   Name1, Name2, ...
WHERE  conditions on attributes of Name1 and Name2

In [ ]:
-- Recursive Queries
-- 示例：计算给定零件中所有子零件的数量。
-- 架构：零件（零件，子零件，数量）
WITH RECURSIVE IncludedParts(sub_part, part,quantity) AS ( 
    SELECT sub_part, part,quantity 
    FROM Parts WHERE part = GivenPart 
    UNION ALL 
    SELECT p.sub_part, p.part, p.quantity 
    FROM IncludedParts i, Parts p 
    WHERE p.part = i .sub_part 
) 
SELECT sub_part, SUM(quantity) as total_quantity 
FROM IncludedParts 
GROUP BY sub_part
-- 包括子零件、子子零件、子子子零件等。

## **SQL Views**

In [ ]:
-- VIEW
CREATE VIEW
    CourseMarksAndAverages(course,term,student,mark,avg)
AS
SELECT s.code, termName(t.id), e.student, e.mark,
    avg(mark) OVER (PARTITION BY course)
FROM   CourseEnrolments e
    JOIN Courses c on c.id = e.course
    JOIN Subjects s on s.id = c.subject
    JOIN Terms t on t.id = c.term;

## **SQL Functions**

In [ ]:
-- Within the function, arguments are accessed as $1, $2, ...
CREATE OR REPLACE
    funcName(arg1type, arg2type, ....)
    RETURNS setof rettype
AS $$
DECLARE
    result integer;
    tup record;
BEGIN
    select balance into result
    from   Accounts
    where  acctNo = acctNum;

    if (not found) then    
        result := 1;  
    elsif (condition) then
        result := 2;
    else
        result := 3;
    end if;

    -- 调试输出
    raise notice 'x+1 = %, y = %, z = %' , x+1, y, z;

    -- 循环
    for i in 1..n loop
        result := result * i;
    end loop;

    FOR tup IN Query 
    LOOP 
        Statements;
        return next tup;
    end loop;

    return result;

exception
    when division_by_zero then
        raise notice 'caught division_by_zero';
        return result;
    when floating_point_exception then
        raise notice 'caught floating_point_exception';
        return result;
END;
$$ LANGUAGE plpgsql;

## **Aggregates**

In [ ]:
-- Aggregates reduce a collection of values into a single result.
CREATE AGGREGATE AggName(BaseType) (
    sfunc     = UpdateStateFunction,
    stype     = StateType,
    initcond  = InitialValue,
    finalfunc = MakeFinalFunction,
    sortop    = OrderingOperator
);
-- initcond（类型StateType）是可选的；默认为NULL
-- finalfunc是可选的；默认为身份函数
-- sortop是可选的；最小/最大类型聚合需要

select string_agg(x,'+') from R;

In [ ]:
-- Example: defining the count aggregate (roughly)
create aggregate myCount(anyelement) (
    stype    = int,    -- the accumulator type
    initcond = 0,      -- initial accumulator value
    sfunc    = oneMore -- increment function
);

create function
    oneMore(sum int, x anyelement) returns int
as $$
begin return sum + 1; end;
$$ language plpgsql;

In [ ]:
-- Example: product aggregate
create function
    mult(soFar numeric, next numeric) returns numeric
as $$
begin return soFar * next; end;
$$ language plpgsql;

create aggregate prod(numeric) (
    stype    = numeric,
    initcond = 1,
    sfunc    = mult
);

## **Assertions (不支持)**

In [ ]:
-- Example: #students in any UNSW course must be < 10000
create assertion ClassSizeConstraint check (
    not exists (
        select c.id
        from   Courses c
        join Enrolments e on (c.id = e.course)
        group  by c.id
        having count(e.student) > 9999
    )
);

In [ ]:
-- Example: assets of branch = sum of its account balances
create assertion AssetsCheck check (
    not exists (
        select branchName from Branches b
        where  b.assets <>
            (select sum(a.balance) from Accounts a
            where a.branch = b.location)
    )
);

## **Triggers**

<img src="https://cgi.cse.unsw.edu.au/~cs3311/21T3/lectures/triggers/Pics/dbms/trigger-seq.png" width="60%">

In [ ]:
CREATE TRIGGER TriggerName
{AFTER|BEFORE}  Event1 [ OR Event2 ... ] -- Possible Events are INSERT, DELETE, UPDATE
ON TableName
[ WHEN ( Condition ) ]
FOR EACH {ROW|STATEMENT}
EXECUTE PROCEDURE FunctionName(args...);

-- However a BEFORE function must contain one of:
RETURN old;    or    RETURN new;

### **Examples**

In [ ]:
-- Case 1: new employees arrive
create trigger TotalSalary1
after insert on Employees
for each row execute procedure totalSalary1();

create function totalSalary1() returns trigger
as $$
begin
    if (new.dept is not null) then
        update Department
        set    totSal = totSal + new.salary
        where  Department.id = new.dept;
    end if;
    return new;
end;
$$ language plpgsql;

In [ ]:
-- Case 2: employees change departments/salaries
create trigger TotalSalary2
after update on Employee
for each row execute procedure totalSalary2();

create function totalSalary2() returns trigger
as $$
begin
    update Department
    set    totSal = totSal + new.salary
    where  Department.id = new.dept;
    update Department
    set    totSal = totSal - old.salary
    where  Department.id = old.dept;
    return new;
end;
$$ language plpgsql;

In [ ]:
-- Case 3: employees leave
create trigger TotalSalary3
after delete on Employee
for each row execute procedure totalSalary3();

create function totalSalary3() returns trigger
as $$
begin
    if (old.dept is not null) then
        update Department
        set    totSal = totSal - old.salary
        where  Department.id = old.dept;
    end if;
    return old;
end;
$$ language plpgsql;

## **psycopg2 示例**

In [ ]:
import psycopg2
# dbname ... 数据库名称
# user... 用户名（用于身份验证）
# password... 用户密码（用于身份验证）
# host...服务器在哪里运行（默认=本地主机）
# port...服务器正在侦听哪个端口（默认值 = 5432）
try:
    db = psycopg2.connect("dbname = ass1")
    cur = db.cursor()
    cur.execute("select name from Beers limit 5")
    for tup in cur.fetchall():  # cur.fetchone(), cur.fetchmany(3)
        x = tup[0]
        print(x)
        
except Exception as e:
    print("Unable to connect to the database\n", e)

else:
    cur.close()
    db.close()

## **Normal Forms**

First,Second,Third Normal Forms (1NF,2NF,3NF) (Codd 1972)  
Boyce-Codd Normal Form (BCNF) (1974)  
Fourth Normal Form (4NF) (Zaniolo 1976, Fagin 1977)  
Fifth Normal Form (5NF) (Fagin 1979)  
我们说“架构在 xNF 中”，这...  

告诉我们一些关于模式中冗余级别的信息  
1NF 允许大部分冗余；5NF 允许最少的冗余。  

对于大多数实际用途，BCNF（或 3NF）是可接受的 NF。  
在实践中，BCNF 和 3NF 是最重要的。  

博伊斯-科德范式 (BCNF)：  

消除了由于功能依赖性导致的所有冗余, 没有更新异常  
但可能不会保留原始的功能依赖  

第三范式（3NF）：  
由于fd s 消除了大部分（但不是全部）冗余  
保证保留所有功能依赖  

## **Relational Algebra**


Core relational algebra operations:  

rename: 更改关系/属性的名称  
selection: 选择元组/行的子集  
projection: 选择属性/列的子集  
union, intersection, difference: 组合关系  
product, join: 组合关系  

<img src="QQ截图20211130121241.png" width="60%">

In [ ]:
-- Beers made by Sierra Nevada
SNBeers = Sel[manf=Sierra Nevada](Beers)
Result  = Rename[beer](Proj[name](SNBeers)
-- Names of all beers
Result = Proj[name](Beers)
-- Names of drinkers who live in Newtown
Result = Proj[name](Sel[addr=Newtown](Drinkers))
-- What are all of the breweries?
Result(brewer) = Proj[manf](Beers)

### **Union and Intersect**
<img src="https://cgi.cse.unsw.edu.au/~cs3311/21T3/lectures/ra-set-ops/Pics/relalg/setops.png" width="60%">

In [ ]:

JohnBars = Proj[bar](Sel[drinker=John](Frequents))
GernotBars = Proj[bar](Sel[drinker=Gernot](Frequents))
-- Bars where either John or Gernot drinks
Result = JohnBars union GernotBars
-- Bars where both John and Gernot drink
Result = JohnBars intersect GernotBars

### **Difference**
<img src="https://cgi.cse.unsw.edu.au/~cs3311/21T3/lectures/ra-set-ops/Pics/relalg/difference2.png" width="60%">  

Clearly, difference is not symmetric.  

In [ ]:
-- Bars where John drinks and Gernot doesn't
JohnBars = Proj[bar](Sel[drinker=John](Frequents))
GernotBars = Proj[bar](Sel[drinker=Gernot](Frequents))
Result = JohnBars - GernotBars

--Bars that sell VB but not New
VBBars = Proj[bar](Sel[beer=VB](Sells))
NewBars = Proj[bar](Sel[beer=New](Sells))
Result = VBBars - NewBars

### **Theta Join**

In [ ]:
-- Who drinks in Newtown bars?
NewtownBars(nbar) = Sel[addr=Newtown](Bars)
Tmp = Frequents Join[bar=nbar] NewtownBars
Result(drinker) = Proj[drinker](Tmp)
-- Who drinks beers made by Carlton?
CarltonBeers = Sel[manf=Carlton](Beers)
Tmp = Likes Join[beer=name] CarltonBeers
Result(drinker) = Proj[drinker]Tmp